In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import linprog

In [2]:
class PSO:
    
    def __init__(self, c, A_ub = None, b_ub = None, bounds = None, args = None, n_particles = 150, maxiter = 500, w=0.7, c1=0.7, c2=0.7):
        self.c = c
        self.A_ub = A_ub
        self.b_ub = b_ub
        self.bounds = bounds
        self.args = args
        self.n_particles = n_particles
        self.maxiter = maxiter
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.dim = len(c)

    def calculate_max_bound(self):
        pos, jump = 0, 100000
        while jump > 0:
            while self.obj_func(np.random.uniform(0, pos, self.dim)) < self.obj_func(np.random.uniform(0, pos+jump, self.dim)):
                pos += jump
            jump/=2
        return pos

    def optimize(self, verbose = False):
        min_bound = np.array([self.bounds[i][0] if self.bounds[i][0] is not None else 0 for i in range(self.bounds.shape[0])])
        max_bound = np.array([self.bounds[i][1] if self.bounds[i][1] is not None else 100 for i in range(self.bounds.shape[0])])
        particles = np.random.uniform(min_bound, max_bound, (self.n_particles, self.dim))
        velocity = np.random.normal(0.01, 0.5, size=(self.n_particles, self.dim))
        gbest = particles[0]
        gbest_fitness = self.obj_func(gbest)

        for it in range(self.maxiter):
            fitness = np.apply_along_axis(self.obj_func, 1, particles)
            
            pbest = particles[fitness.argmin()]
            if fitness.min() < gbest_fitness or it == 0:
                gbest_fitness = fitness.min()
                gbest = pbest

            new_vel = (self.w * velocity) + (self.c1 * np.random.uniform(0, 1, (self.n_particles, self.dim)) * (pbest - particles)) + (self.c2 * np.random.uniform(0, 1, (self.n_particles, self.dim)) * (gbest - particles))

            velocity = new_vel
            particles = particles + new_vel

            if self.c1c2w_update_func is not None:
                self.c1, self.c2, self.w = self.c1c2w_update_func(it, self.maxiter, self.c1, self.c2, self.w)

            if verbose:
                print("Iteration: {}, Best fitness: {}".format(it, gbest_fitness))

        return gbest, gbest_fitness

    def c1c2w_update_func(self, it, maxiter, c1, c2, w):
        return c1*0.99, c2*0.99, w*0.99

    def obj_func(self, x):
        fitness = np.dot(self.c, x)
        if self.A_ub is not None and self.b_ub is not None:
            assert self.A_ub.shape[1] == x.shape[0], "A_ub and x must have the same number of columns"
            assert self.A_ub.shape[0] == self.b_ub.shape[0], "A_ub and b_ub must have the same number of rows"
            for i in range(len(self.A_ub)):
                result = np.dot(self.A_ub[i], x)
                if result > self.b_ub[i]:
                    fitness += 100000000
        if self.bounds is not None:
            for i in range(len(x)):
                if (self.bounds[i][0] != None and x[i] < self.bounds[i][0]) or (self.bounds[i][1] != None and x[i] > self.bounds[i][1]):
                    fitness += 100000000
        return fitness

$ Bicicletas $ <br>
$ \max_{p, m} 20000p + 15000m $ <br>
$ p + 2m \le 80 $ <br>
$ 3p + 2m \le 120 $ <br>
$ p, m \ge 0 $

In [3]:
c = [-20000, -15000]
A = [[1, 2], [3, 2]]
b = [80, 120]
p_bounds = (0, None)
m_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[p_bounds, m_bounds])
print(res.x)

[20. 30.]


In [4]:
pso = PSO(c, A_ub=np.array(A), b_ub=np.array(b), bounds=np.array([p_bounds, m_bounds]))
gbest, gbest_fitness = pso.optimize(verbose = False)
print(gbest, gbest_fitness)

[20. 30.] -849999.9999999168


$ Lamparas $ <br>
$ \max_{ l_{1}, l_{2}} 15l_{1} + 10l_{2} $ <br>
$ 20l_{1} + 30l_{2} \le 100 $ <br>
$ 10l_{1} + 10l_{2} \le 80 $ <br>
$ l_{1}, l_{2} \ge 0 $

In [5]:
c = [-15, -10]
A = [[20, 30], [10, 10]]
b = [100, 80]
l1_bounds = (0, None)
l2_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[l1_bounds, l2_bounds])
print(res.x)

[5. 0.]


In [6]:
pso = PSO(c, A_ub=np.array(A), b_ub=np.array(b), bounds=np.array([l1_bounds, l2_bounds]))
gbest, gbest_fitness = pso.optimize(verbose = False)
print(gbest, gbest_fitness)

[3591.11794417  203.81440836] 199944095.0867539


$ Granja-de-pollos $ <br>
$ \min_{ x, y} 10x + 30y $ <br>
$ x + 5y \ge 15 $ <br>
$ 5x + y \ge 15 $ <br>
$ x,y \ge 0 $

In [7]:
c = [10, 30]
A = [[-1, -5], [-5, -1]]
b = [-15, -15]
x_bounds = (0, None)
y_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds])
print(res.x)

[2.5 2.5]


In [8]:
pso = PSO(c, A_ub=np.array(A), b_ub=np.array(b), bounds=np.array([x_bounds, y_bounds]))
gbest, gbest_fitness = pso.optimize(verbose = False)
print(gbest, gbest_fitness)

[2.5 2.5] 100.0000000000012


$ Ofertas $ <br>
$ \max_{ x,y } 6.5x + 7y $ <br>
$ 2x + 3y \le 600 $ <br>
$ x + y \le 500 $ <br>
$ 2x + y \le 400 $ <br>
$ x,y \ge 0 $

In [9]:
c = [-6.5, -7]
A = [[2, 3], [1, 1], [2,1]]
b = [600 , 500, 400]
x_bounds = (0, None)
y_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds])
print(res.x)

[150. 100.]


In [10]:
pso = PSO(c, A_ub=np.array(A), b_ub=np.array(b), bounds=np.array([x_bounds, y_bounds]))
gbest, gbest_fitness = pso.optimize(verbose = False)
print(gbest, gbest_fitness)

[150. 100.] -1674.9999999999984


$ Autobuses $ <br>
$ \min_{ a,b} 800a + 600b $ <br>
$ 50a + 40b \ge 400 $ <br>
$ a + b \le 9 $ <br>
$ 0 \le a \le 10 $ <br>
$ 0 \le b \le 8 $ <br>

In [11]:
c = [800, 600]
A = [[-50, -40], [1,1]]
b = [-400, 9]
x_bounds = (0, 10)
y_bounds = (0, 8)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds])
print(res.x)

[4. 5.]


In [12]:
pso = PSO(c, A_ub=np.array(A), b_ub=np.array(b), bounds=np.array([x_bounds, y_bounds]))
gbest, gbest_fitness = pso.optimize(verbose = False)
print(gbest, gbest_fitness)

[4.00123077 4.99846154] 6200.061538310799
